In [30]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense , Dropout
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score , GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix , accuracy_score
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
# mlflow.set_tracking_uri('http://localhost:5000/')
# ann = mlflow.set_experiment('ANN-bank')

In [6]:
%store -r df

In [7]:
%store -r X

In [8]:
y = df[['Exited']]

In [9]:
X.shape , y.shape

((10000, 13), (10000, 1))

In [10]:
def split_data(X ,y):    
    X_train ,  x_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    x_test = sc.fit_transform(x_test)
    return X_train , x_test , y_train , y_test

In [22]:
X_train , x_test , y_train , y_test = split_data(X, y)

In [23]:
X_train.shape , x_test.shape , y_train.shape , y_test.shape

((8000, 13), (2000, 13), (8000, 1), (2000, 1))

In [28]:
def build_classifier(X_train,y_train,x_test,y_test):


        classifier = Sequential()
        classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu' , input_dim=13))
        classifier.add(Dropout(p=0.1))
        classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu'))
        classifier.add(Dropout(p=0.1))
        classifier.add(Dense(output_dim=1 , kernel_initializer='uniform' , activation='sigmoid'))
        classifier.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])

        classifier.fit(X_train,y_train, batch_size=10 , epochs=100, verbose=2 , callbacks=[tensorboard])
        y_pred = classifier.predict(x_test)
        y_pred = y_pred > 0.5
        cm = confusion_matrix(y_test,y_pred)
        accuracy = accuracy_score(y_test , y_pred) * 100

In [29]:
build_classifier(X_train , y_train , x_test, y_test)

/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", input_dim=13, units=6)`
  """
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=6)`
  import sys
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="sigmoid", units=1)`
  if __name__ == '__main__':


Epoch 1/100
 - 1s - loss: 0.4915 - accuracy: 0.7958
Epoch 2/100
 - 1s - loss: 0.4358 - accuracy: 0.7960
Epoch 3/100
 - 1s - loss: 0.4317 - accuracy: 0.7960
Epoch 4/100
 - 1s - loss: 0.4291 - accuracy: 0.7960
Epoch 5/100
 - 1s - loss: 0.4281 - accuracy: 0.8037
Epoch 6/100
 - 1s - loss: 0.4279 - accuracy: 0.8204
Epoch 7/100
 - 1s - loss: 0.4245 - accuracy: 0.8260
Epoch 8/100
 - 1s - loss: 0.4237 - accuracy: 0.8279
Epoch 9/100
 - 1s - loss: 0.4257 - accuracy: 0.8284
Epoch 10/100
 - 1s - loss: 0.4237 - accuracy: 0.8282
Epoch 11/100
 - 1s - loss: 0.4232 - accuracy: 0.8271
Epoch 12/100
 - 1s - loss: 0.4224 - accuracy: 0.8307
Epoch 13/100
 - 1s - loss: 0.4233 - accuracy: 0.8305
Epoch 14/100
 - 1s - loss: 0.4208 - accuracy: 0.8295
Epoch 15/100
 - 1s - loss: 0.4239 - accuracy: 0.8288
Epoch 16/100
 - 1s - loss: 0.4201 - accuracy: 0.8300
Epoch 17/100
 - 1s - loss: 0.4221 - accuracy: 0.8296
Epoch 18/100
 - 1s - loss: 0.4223 - accuracy: 0.8296
Epoch 19/100
 - 1s - loss: 0.4213 - accuracy: 0.8300
Ep

In [12]:
def build_kfold_classifier():
        classifier_kfold= Sequential()
        classifier_kfold.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu' , input_dim=13))
        classifier_kfold.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu'))
        classifier_kfold.add(Dense(output_dim=1 , kernel_initializer='uniform' , activation='sigmoid'))
        classifier_kfold.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])
        return classifier_kfold

In [13]:
classifier_kfold_wrapper = KerasClassifier(build_fn=build_kfold_classifier , 
                                                   batch_size=10 , epochs=20, verbose=2)
accu = cross_val_score(estimator=classifier_kfold_wrapper , 
                                X=X_train , y=y_train , cv=10 , verbose=1 , n_jobs=-1)
        
       
        
        


    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.4min finished


In [14]:
accu

array([0.85874999, 0.83749998, 0.82875001, 0.82875001, 0.85624999,
       0.83375001, 0.84125   , 0.82499999, 0.81124997, 0.85500002])

In [17]:
mean = accu.mean()

In [19]:
variance = accu.std()

In [20]:
variance , mean

(0.014614315297194522, 0.8376249969005585)

In [33]:
def grid_search(optimizer):
        classifier = Sequential()
        classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu' , input_dim=13))
        classifier.add(Dropout(p=0.1))
        classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu'))
        classifier.add(Dropout(p=0.1))
        classifier.add(Dense(output_dim=1 , kernel_initializer='uniform' , activation='sigmoid'))
        classifier.compile(optimizer=optimizer, loss='binary_crossentropy' , metrics=['accuracy'])
        return classifier

In [36]:
classifier_grid_search = KerasClassifier(build_fn=grid_search)

In [37]:
parameters = {'batch_size': [25,32] , 
               'nb_epoch': [100,500],
               'optimizer': ['adam','rmsprop']}

In [41]:
grid_search = GridSearchCV(estimator=classifier_grid_search ,
                           param_grid=parameters , 
                           scoring='accuracy',
                           cv=2).fit(X_train , y_train)

/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", input_dim=13, units=6)`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=6)`
  """
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2

Epoch 1/1
4000/4000 [==============================] - 1s 134us/step - loss: 0.6214 - accuracy: 0.7960
Epoch 1/1
4000/4000 [==============================] - 1s 139us/step - loss: 0.6476 - accuracy: 0.7918
Epoch 1/1
4000/4000 [==============================] - 0s 123us/step - loss: 0.6224 - accuracy: 0.7970
Epoch 1/1
4000/4000 [==============================] - 0s 96us/step - loss: 0.6585 - accuracy: 0.7908
Epoch 1/1
4000/4000 [==============================] - 0s 115us/step - loss: 0.6393 - accuracy: 0.7928
Epoch 1/1
4000/4000 [==============================] - 0s 118us/step - loss: 0.6387 - accuracy: 0.7918
Epoch 1/1
4000/4000 [==============================] - 0s 105us/step - loss: 0.6197 - accuracy: 0.7970
Epoch 1/1
4000/4000 [==============================] - 0s 100us/step - loss: 0.6463 - accuracy: 0.7930
Epoch 1/1
4000/4000 [==============================] - 0s 101us/step - loss: 0.6709 - accuracy: 0.7940
Epoch 1/1
4000/4000 [==============================] - 0s 100us/step - los

In [43]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [44]:
best_parameters

{'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'adam'}

In [45]:
best_accuracy

0.796